<a href="https://colab.research.google.com/github/dianamarchila/dianamarchila.github.io/blob/main/DataModelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proyecto predicción de la reserva de dinero para reclamaciones en empresas de seguros**

**Data Modelling**

Realizado por: Diana Archila Cordero

In [ ]:
# Configuración para ejecutar código de R
!pip install rpy2==3.5.1
%reload_ext rpy2.ipython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rpy2: filename=rpy2-3.5.1-cp310-cp310-linux_x86_64.whl size=314924 sha256=da4ce6cf37a9dfad2bb711b935db88392b0eb2a0bc57b23b07a77f6d8d0ab3ff
  Stored in directory: /root/.cache/pip/wheels/73/a6/ff/4e75dd1ce1cfa2b9a670cbccf6a1e41c553199e9b25f05d953
Successfully built rpy2
  Attempting uninstall: rpy2
    Found existing installation: rpy2 3.4.2
    Uninstalling rpy2-3.4.2:
      Successfully uninstalled rpy2-3.4.2


In [ ]:
%%R # Comando que indica que la celda contiene código de R
install.packages("dplyr")
require(dplyr)

# Cargue del dataset
data <- read.csv("/content/drive/MyDrive/ML /medmal_pos.csv") %>% filter(GRCODE %in% c(669, 683, 7854, 32514, 33049, 33111, 36072, 36234, 36277, 36676, 40568, 40975, 41467, 43656, 43770, 44504))
# Se excluyen aquellas empresas que tienen en 0 todos los valores de la matriz

colnames(data)


Attaching package: ‘dplyr’



    filter, lag



    intersect, setdiff, setequal, union




 [1] "GRCODE"             "GRNAME"             "AccidentYear"      
 [4] "DevelopmentYear"    "DevelopmentLag"     "IncurLoss_F2"      
 [7] "CumPaidLoss_F2"     "BulkLoss_F2"        "EarnedPremDIR_F2"  
[10] "EarnedPremCeded_F2" "EarnedPremNet_F2"   "Single"            
[13] "PostedReserve97_F2"


## Datos de validación de desempeño de los modelos

Se construyen las matrices de cada empresa, de manera que esta es la información que se utilizará para evaluar el desempeño de cada uno de los modelos aplicados. Estas matrices únicamente se usarán para las pruebas de cada modelo, más no como parte del entrenamiento de los mismos.

In [ ]:
%%R
require(dplyr)

# Construcción de matrices de cada empresa, Variable: CumPaidLoss_F2
real_data <- data %>% mutate(
                   t0 = case_when(DevelopmentLag == 1 ~ CumPaidLoss_F2),
                   t1 = case_when(DevelopmentLag == 2 ~ CumPaidLoss_F2),
                   t2 = case_when(DevelopmentLag == 3 ~ CumPaidLoss_F2),
                   t3 = case_when(DevelopmentLag == 4 ~ CumPaidLoss_F2),
                   t4 = case_when(DevelopmentLag == 5 ~ CumPaidLoss_F2),
                   t5 = case_when(DevelopmentLag == 6 ~ CumPaidLoss_F2),
                   t6 = case_when(DevelopmentLag == 7 ~ CumPaidLoss_F2),
                   t7 = case_when(DevelopmentLag == 8 ~ CumPaidLoss_F2),
                   t8 = case_when(DevelopmentLag == 9 ~ CumPaidLoss_F2),
                   t9 = case_when(DevelopmentLag == 10 ~ CumPaidLoss_F2)
                   )
real_data <- real_data %>% group_by(GRCODE, AccidentYear) %>%
          summarise(t0 = sum(t0, na.rm = T),
                    t1 = sum(t1, na.rm = T),
                    t2 = sum(t2, na.rm = T),
                    t3 = sum(t3, na.rm = T),
                    t4 = sum(t4, na.rm = T),
                    t5 = sum(t5, na.rm = T),
                    t6 = sum(t6, na.rm = T),
                    t7 = sum(t7, na.rm = T),
                    t8 = sum(t8, na.rm = T),
                    t9 = sum(t9, na.rm = T))

print(head(real_data, 10))

`summarise()` has grouped output by 'GRCODE'. You can override using the
`.groups` argument.
# A tibble: 10 × 12
# Groups:   GRCODE [1]
   GRCODE AccidentYear    t0    t1    t2    t3    t4    t5    t6    t7    t8
    <int>        <int> <int> <int> <int> <int> <int> <int> <int> <int> <int>
 1    669         1988  2716 24576 43990 59722 71019 76354 76792 77207 77588
 2    669         1989  3835 25158 45145 60331 67457 70821 71769 72085 72035
 3    669         1990  4838 27965 50873 66400 71875 74755 75176 75250 75250
 4    669         1991  4456 34241 64737 79390 84465 87375 89119 89825 90333
 5    669         1992  5970 36080 68268 81783 86076 87167 88282 88857 89087
 6    669         1993  9398 46210 77045 86298 91796 93827 95871 95943 95987
 7    669         1994  6181 39204 70006 82385 86523 88387 90408 90525 91640
 8    669         1995  7828 42356 70729 79340 81142 81891 81905 81897 81781
 9    669         1996  8854 51400 81653 90504 94284 96456 97305 97175 97212
10    669        

## Información inicial

El entrenamiento de los modelos, en todos los casos se hará con los triángulos superiores de la matriz propia de cada empresa, es decir donde el año de desarrollo del accidente es menor o igual a 1997.

In [ ]:
%%R
require(dplyr)

# Construcción de triángulos de cada empresa, Variable: CumPaidLoss_F2
triang_data <- data %>% filter(DevelopmentYear<=1997) %>%
            mutate(CumPaidLoss_F2 = as.double(CumPaidLoss_F2),
                   t0 = case_when(DevelopmentLag == 1 ~ CumPaidLoss_F2),
                   t1 = case_when(DevelopmentLag == 2 ~ CumPaidLoss_F2),
                   t2 = case_when(DevelopmentLag == 3 ~ CumPaidLoss_F2),
                   t3 = case_when(DevelopmentLag == 4 ~ CumPaidLoss_F2),
                   t4 = case_when(DevelopmentLag == 5 ~ CumPaidLoss_F2),
                   t5 = case_when(DevelopmentLag == 6 ~ CumPaidLoss_F2),
                   t6 = case_when(DevelopmentLag == 7 ~ CumPaidLoss_F2),
                   t7 = case_when(DevelopmentLag == 8 ~ CumPaidLoss_F2),
                   t8 = case_when(DevelopmentLag == 9 ~ CumPaidLoss_F2),
                   t9 = case_when(DevelopmentLag == 10 ~ CumPaidLoss_F2)
                   )
triang_data <- triang_data %>% group_by(GRCODE, AccidentYear) %>%
          summarise(t0 = sum(t0, na.rm = T),
                    t1 = sum(t1, na.rm = T),
                    t2 = sum(t2, na.rm = T),
                    t3 = sum(t3, na.rm = T),
                    t4 = sum(t4, na.rm = T),
                    t5 = sum(t5, na.rm = T),
                    t6 = sum(t6, na.rm = T),
                    t7 = sum(t7, na.rm = T),
                    t8 = sum(t8, na.rm = T),
                    t9 = sum(t9, na.rm = T)
                    )

print(head(triang_data, 100))

`summarise()` has grouped output by 'GRCODE'. You can override using the
`.groups` argument.
# A tibble: 100 × 12
# Groups:   GRCODE [10]
   GRCODE AccidentYear    t0    t1    t2    t3    t4    t5    t6    t7    t8
    <int>        <int> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
 1    669         1988  2716 24576 43990 59722 71019 76354 76792 77207 77588
 2    669         1989  3835 25158 45145 60331 67457 70821 71769 72085 72035
 3    669         1990  4838 27965 50873 66400 71875 74755 75176 75250     0
 4    669         1991  4456 34241 64737 79390 84465 87375 89119     0     0
 5    669         1992  5970 36080 68268 81783 86076 87167     0     0     0
 6    669         1993  9398 46210 77045 86298 91796     0     0     0     0
 7    669         1994  6181 39204 70006 82385     0     0     0     0     0
 8    669         1995  7828 42356 70729     0     0     0     0     0     0
 9    669         1996  8854 51400     0     0     0     0     0     0     0
10    669      

## Modelo Chain-Ladder Determinista

In [ ]:
%%R
require(dplyr)

# Estructura de entrenamiento y prueba de los modelos
comp_codes <- data %>% group_by(GRCODE) %>% tally()

# Información de desempeño del modelo
perf_model <- data.frame()

for(i in 1:nrow(comp_codes)){
    comp_code_i <- comp_codes[i, 1]

    # Exclusión de la matriz de prueba
    train_data <- triang_data %>% filter(GRCODE == comp_code_i)

    # Entrenamiento del modelo. Cálculo de factores de desarrollo
    dev_fact <- c()

    dev_fact_0 <- sum((train_data %>% filter(AccidentYear <= 1997-1))$t1)/sum((train_data %>% filter(AccidentYear <= 1997-1))$t0)
    dev_fact_1 <- sum((train_data %>% filter(AccidentYear <= 1997-2))$t2)/sum((train_data %>% filter(AccidentYear <= 1997-2))$t1)
    dev_fact_2 <- sum((train_data %>% filter(AccidentYear <= 1997-3))$t3)/sum((train_data %>% filter(AccidentYear <= 1997-3))$t2)
    dev_fact_3 <- sum((train_data %>% filter(AccidentYear <= 1997-4))$t4)/sum((train_data %>% filter(AccidentYear <= 1997-4))$t3)
    dev_fact_4 <- sum((train_data %>% filter(AccidentYear <= 1997-5))$t5)/sum((train_data %>% filter(AccidentYear <= 1997-5))$t4)
    dev_fact_5 <- sum((train_data %>% filter(AccidentYear <= 1997-6))$t6)/sum((train_data %>% filter(AccidentYear <= 1997-6))$t5)
    dev_fact_6 <- sum((train_data %>% filter(AccidentYear <= 1997-7))$t7)/sum((train_data %>% filter(AccidentYear <= 1997-7))$t6)
    dev_fact_7 <- sum((train_data %>% filter(AccidentYear <= 1997-8))$t8)/sum((train_data %>% filter(AccidentYear <= 1997-8))$t7)
    dev_fact_8 <- sum((train_data %>% filter(AccidentYear <= 1997-9))$t9)/sum((train_data %>% filter(AccidentYear <= 1997-9))$t8)

    dev_fact <- c(dev_fact_0,
                  dev_fact_1,
                  dev_fact_2,
                  dev_fact_3,
                  dev_fact_4,
                  dev_fact_5,
                  dev_fact_6,
                  dev_fact_7,
                  dev_fact_8)

    # Prueba del modelo
    test <- train_data[3:ncol(train_data)]
    real <- real_data %>% filter(GRCODE == comp_code_i)
    real <- real[3:ncol(real)]

    se <- c()
    re <- c()

    for(j in 2:10){
        for(k in (ncol(test)-j+2):ncol(test)){
            test[j, k] = (test[j, (k-1)] * dev_fact[k-1])

            dif <- as.double((real[j, k] - test[j, k]))

            se <- c(se, as.double((dif^2)))
            re <- c(re, as.double((abs(dif)/real[j, k])*100))
        }
    }

    MSE <- mean(se)
    MAPE <- mean(re)

    res_ml <- as.data.frame(c(comp_code_i, MSE, MAPE, 'CL_Det'))
    colnames(res_ml) <- c('train_code', 'MSE', 'MAPE', 'model')

    perf_model <- rbind(perf_model, res_ml)
}

print(perf_model)
print(mean(perf_model$MAPE, na.rm=TRUE))

   train_code          MSE      MAPE  model
1         669 2.416102e+08 11.861763 CL_Det
2         683 4.536666e+07 16.109026 CL_Det
3        7854 1.430287e+07 11.560831 CL_Det
4       32514 9.688122e+06 45.388821 CL_Det
5       33049 1.019248e+08 14.065446 CL_Det
6       33111 5.931404e+07 26.303909 CL_Det
7       36072          NaN       NaN CL_Det
8       36234 1.958745e+06 12.089391 CL_Det
9       36277 2.461716e+06 12.082369 CL_Det
10      36676 7.201303e+05  4.527065 CL_Det
11      40568 6.292751e+05 17.936511 CL_Det
12      40975 4.251034e+06 13.820234 CL_Det
13      41467 3.661659e+09 45.433572 CL_Det
14      43656 1.678206e+06 10.731029 CL_Det
15      43770 7.439463e+04 16.253352 CL_Det
16      44504          NaN       NaN CL_Det
[1] 18.44024


# Leave One Out Cross Validation

In [ ]:
%%R
require(dplyr)

# Estructura de ajuste y validación de los modelos
comp_codes <- data %>% group_by(GRCODE) %>% tally()

# Información de desempeño del modelo
perf_model <- data.frame()

for(i in 1:nrow(comp_codes)){
    comp_code_i <- comp_codes[i, 1]

    # Exclusión de la matriz de prueba
    train_data <- triang_data %>% filter(GRCODE != comp_code_i)
    test_data <- triang_data %>% filter(GRCODE == comp_code_i)

    # Entrenamiento del modelo. Cálculo de factores de desarrollo
    dev_fact <- c()

    dev_fact_0 <- sum((train_data %>% filter(AccidentYear <= 1997-1))$t1)/sum((train_data %>% filter(AccidentYear <= 1997-1))$t0)
    dev_fact_1 <- sum((train_data %>% filter(AccidentYear <= 1997-2))$t2)/sum((train_data %>% filter(AccidentYear <= 1997-2))$t1)
    dev_fact_2 <- sum((train_data %>% filter(AccidentYear <= 1997-3))$t3)/sum((train_data %>% filter(AccidentYear <= 1997-3))$t2)
    dev_fact_3 <- sum((train_data %>% filter(AccidentYear <= 1997-4))$t4)/sum((train_data %>% filter(AccidentYear <= 1997-4))$t3)
    dev_fact_4 <- sum((train_data %>% filter(AccidentYear <= 1997-5))$t5)/sum((train_data %>% filter(AccidentYear <= 1997-5))$t4)
    dev_fact_5 <- sum((train_data %>% filter(AccidentYear <= 1997-6))$t6)/sum((train_data %>% filter(AccidentYear <= 1997-6))$t5)
    dev_fact_6 <- sum((train_data %>% filter(AccidentYear <= 1997-7))$t7)/sum((train_data %>% filter(AccidentYear <= 1997-7))$t6)
    dev_fact_7 <- sum((train_data %>% filter(AccidentYear <= 1997-8))$t8)/sum((train_data %>% filter(AccidentYear <= 1997-8))$t7)
    dev_fact_8 <- sum((train_data %>% filter(AccidentYear <= 1997-9))$t9)/sum((train_data %>% filter(AccidentYear <= 1997-9))$t8)

    dev_fact <- c(dev_fact_0,
                  dev_fact_1,
                  dev_fact_2,
                  dev_fact_3,
                  dev_fact_4,
                  dev_fact_5,
                  dev_fact_6,
                  dev_fact_7,
                  dev_fact_8)

    # Prueba del modelo
    test <- test_data[3:ncol(test_data)]
    real <- real_data %>% filter(GRCODE == comp_code_i)
    real <- real[3:ncol(real)]

    se <- c()
    re <- c()

    for(j in 2:10){
        for(k in (ncol(test)-j+2):ncol(test)){
            test[j, k] = (test[j, (k-1)] * dev_fact[k-1])

            dif <- as.double((real[j, k] - test[j, k]))

            se <- c(se, as.double((dif^2)))
            re <- c(re, as.double((abs(dif)/real[j, k])*100))
        }
    }

    MSE <- mean(se, na.rm = TRUE)
    MAPE <- mean(re, na.rm = TRUE)

    res_ml <- as.data.frame(c(comp_code_i, MSE, MAPE, 'CL_Det'))
    colnames(res_ml) <- c('train_code', 'MSE', 'MAPE', 'model')

    perf_model <- rbind(perf_model, res_ml)
}

print(perf_model)
print(mean(perf_model$MAPE, na.rm=T))

   train_code          MSE     MAPE  model
1         669 7249807065.9 68.68239 CL_Det
2         683   86037614.1 23.86461 CL_Det
3        7854   26013134.2 20.15882 CL_Det
4       32514    4581602.8 34.77853 CL_Det
5       33049  247496962.1 20.94825 CL_Det
6       33111   70836499.9 30.70797 CL_Det
7       36072     456718.7 22.62963 CL_Det
8       36234    9450997.5 22.54183 CL_Det
9       36277    2225359.2 13.25483 CL_Det
10      36676   11526453.0 15.55941 CL_Det
11      40568     407197.9 16.17317 CL_Det
12      40975  114630940.7 79.42195 CL_Det
13      41467  834281874.3 30.12851 CL_Det
14      43656    3256000.4 14.80604 CL_Det
15      43770     521891.3 40.00757 CL_Det
16      44504    1183474.9 21.02839 CL_Det
[1] 29.66824


Para poder llevar a cabo la ejecución de Chain Ladder como modelo de regresión, calculamos la matriz de vectores como se indicó en el paper, a esta matriz, se le asignará un valor y, correspondiente a cada valor del triángulo de observaciones.

In [ ]:
import pandas as pd

# Number of rows and columns
n = 10

# Create an empty dataframe with the desired columns
columns = ['intercept'] + [f'alpha_{i}' for i in range(2, n+1)] + [f'beta_{i}' for i in range(2, n+1)]
design_matrix = pd.DataFrame(0, index=[f'y_{i}{j-i+1}' for i in range(1, n+1) for j in range(i, n+1)], columns=columns)

# Populate the dataframe
for row in design_matrix.index:
    i, j = int(row[2]), int(row[3:])
    design_matrix.at[row, 'intercept'] = 1
    if i > 1:
        design_matrix.at[row, f'alpha_{i}'] = 1
    if j > 1 and j != i:  # Add a condition to check if j is not equal to i
        design_matrix.at[row, f'beta_{j}'] = 1

print(design_matrix)
design_matrix.to_csv('matriz_reg.csv', index=False)

       intercept  alpha_2  alpha_3  alpha_4  alpha_5  alpha_6  alpha_7  \
y_11           1        0        0        0        0        0        0   
y_12           1        0        0        0        0        0        0   
y_13           1        0        0        0        0        0        0   
y_14           1        0        0        0        0        0        0   
y_15           1        0        0        0        0        0        0   
y_16           1        0        0        0        0        0        0   
y_17           1        0        0        0        0        0        0   
y_18           1        0        0        0        0        0        0   
y_19           1        0        0        0        0        0        0   
y_110          1        0        0        0        0        0        0   
y_21           1        1        0        0        0        0        0   
y_22           1        1        0        0        0        0        0   
y_23           1        1        0    

Una vez tenemos la matriz, se asignan los respectivos valores de datos y se realiza la regresión.

In [ ]:
%%R
require(dplyr)
design_matrix <- read.csv('/content/matriz_reg.csv')

# Estructura de ajuste y validación de los modelos
comp_codes <- data %>% group_by(GRCODE) %>% tally()

# Información de desempeño del modelo
matr_reg <- data.frame()
matr_ent <- data.frame()

#for(i in 1:nrow(comp_codes)){
    comp_code_i <- comp_codes[1, 1]

    # Entidad a la que se agregará la matrix
    y <- as.vector(triang_data %>% filter(GRCODE == comp_code_i) %>% select(-c(GRCODE, AccidentYear)))

    # Asignar los y a la matriz
    #matr_ent <- cbind(design_matrix,y) %>% mutate(GRCODE = comp_code_i)

    # Volver todas las matrices una
    #matr_reg <- rbind(matr_reg,matr_ent)
#}

print(y)

Adding missing grouping variables: `GRCODE`
$GRCODE
 [1] 669 669 669 669 669 669 669 669 669 669

$t0
 [1] 2716 3835 4838 4456 5970 9398 6181 7828 8854 7818

$t1
 [1] 24576 25158 27965 34241 36080 46210 39204 42356 51400     0

$t2
 [1] 43990 45145 50873 64737 68268 77045 70006 70729     0     0

$t3
 [1] 59722 60331 66400 79390 81783 86298 82385     0     0     0

$t4
 [1] 71019 67457 71875 84465 86076 91796     0     0     0     0

$t5
 [1] 76354 70821 74755 87375 87167     0     0     0     0     0

$t6
 [1] 76792 71769 75176 89119     0     0     0     0     0     0

$t7
 [1] 77207 72085 75250     0     0     0     0     0     0     0

$t8
 [1] 77588 72035     0     0     0     0     0     0     0     0

$t9
 [1] 77656     0     0     0     0     0     0     0     0     0

attr(,"groups")
# A tibble: 1 × 2
  GRCODE       .rows
   <int> <list<int>>
1    669        [10]
